In [265]:
#May require to pip install netCDF4

Documentation: http://www.odip.org/documents/odip/downloads/20/argo-dm-user-manual.pdf

https://archimer.ifremer.fr/doc/00187/29825/68654.pdf 

NetCDF: https://unidata.github.io/netcdf4-python/netCDF4/index.html

In [29]:
import pandas as pd
import numpy as np
import netCDF4

In [309]:
from ftplib import FTP

with FTP("usgodae.org") as ftp:
    ftp.login()
    ftp.cwd('/pub/outgoing/argo/geo/atlantic_ocean/2009/01')
    print(ftp.nlst()[:2])
    x = ftp.nlst()[0]
    print(type(x))
    print(x.split("_")[0])
    ftp.quit()

['20090101_prof.nc', '20090102_prof.nc']
<class 'str'>
20090101


In [1]:
import os
directory = r'C:\Users\Kik\Documents\GitHub\argo_ocean_data_analysis\helper'
for file in os.listdir(directory)[:2]:
    filename = file
    print(filename)

20111202_prof.nc
20111203_prof.nc


In [24]:
# commented out for later etl to download straight from ftp, open, and push to db
# url='ftp://usgodae.org/pub/outgoing/argo/geo/atlantic_ocean/2020/08/20200801_prof.nc'
# f = urllib.request.urlopen(url)

# turns out you have to use netcdf4 - this is a multidimensional dataset, so flat array packages like xarray doesn't work

argo = netCDF4.Dataset('20111203_prof.nc')
argo.variables


{'DATA_TYPE': <class 'netCDF4._netCDF4.Variable'>
 |S1 DATA_TYPE(STRING16)
     long_name: Data type
     conventions: Argo reference table 1
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (16,)
 filling on,
 'FORMAT_VERSION': <class 'netCDF4._netCDF4.Variable'>
 |S1 FORMAT_VERSION(STRING4)
     long_name: File format version
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (4,)
 filling on,
 'HANDBOOK_VERSION': <class 'netCDF4._netCDF4.Variable'>
 |S1 HANDBOOK_VERSION(STRING4)
     long_name: Data handbook version
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (4,)
 filling on,
 'REFERENCE_DATE_TIME': <class 'netCDF4._netCDF4.Variable'>
 |S1 REFERENCE_DATE_TIME(DATE_TIME)
     long_name: Date of reference for Julian days
     conventions: YYYYMMDDHHMISS
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (14,)
 filling on,
 'DATE_CREATION': <class 'netCDF4._netCDF4.Variable'>
 |S1 DATE_CREATION(DATE_TIME)
     long_name: Date of

In [14]:
for key in list(argo.variables.keys()):
    print(list(argo.variables.keys()).index(key), argo.variables[key][:].dtype, argo.variables[key][:].shape, key)

0 |S1 (16,) DATA_TYPE
1 |S1 (4,) FORMAT_VERSION
2 |S1 (4,) HANDBOOK_VERSION
3 |S1 (14,) REFERENCE_DATE_TIME
4 |S1 (14,) DATE_CREATION
5 |S1 (14,) DATE_UPDATE
6 |S1 (79, 8) PLATFORM_NUMBER
7 |S1 (79, 64) PROJECT_NAME
8 |S1 (79, 64) PI_NAME
9 |S1 (79, 4, 16) STATION_PARAMETERS
10 int32 (79,) CYCLE_NUMBER
11 |S1 (79,) DIRECTION
12 |S1 (79, 2) DATA_CENTRE
13 |S1 (79, 32) DC_REFERENCE
14 |S1 (79, 4) DATA_STATE_INDICATOR
15 |S1 (79,) DATA_MODE
16 |S1 (79, 32) PLATFORM_TYPE
17 |S1 (79, 32) FLOAT_SERIAL_NO
18 |S1 (79, 32) FIRMWARE_VERSION
19 |S1 (79, 4) WMO_INST_TYPE
20 float64 (79,) JULD
21 |S1 (79,) JULD_QC
22 float64 (79,) JULD_LOCATION
23 float64 (79,) LATITUDE
24 float64 (79,) LONGITUDE
25 |S1 (79,) POSITION_QC
26 |S1 (79, 8) POSITIONING_SYSTEM
27 |S1 (79,) PROFILE_CNDC_QC
28 |S1 (79,) PROFILE_PRES_QC
29 |S1 (79,) PROFILE_PSAL_QC
30 |S1 (79,) PROFILE_TEMP_QC
31 |S1 (79, 256) VERTICAL_SAMPLING_SCHEME
32 int32 (79,) CONFIG_MISSION_NUMBER
33 float32 (79, 968) CNDC
34 |S1 (79, 968) CNDC_QC
35

C:\Users\Kik\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
C:\Users\Kik\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


In [25]:
def utf_decoding(array):
    return [x.decode('UTF-8') for x in array.data]

def masked_arrays_decoding(masked_array):
    lst = []
    for idx, row in enumerate(masked_array):
        number = ''.join(utf_decoding(row)).strip()
        lst.append(number)
    return lst

def param_masked_arrays_decoding(masked_array):
    # TODO: improve this horrendous function. The scientific calibrations have 4 dimensions which complicate a bit the automation. 
    calibration = []
    for arrays in range(len(masked_array)):
        lst = []
        for idx, row in enumerate(masked_array[arrays].data):
            temp = []
            for x in range(len(row)):
                temp.append(''.join([y.decode('UTF-8') for y in row[x]]).strip())
            lst.append(temp)
        calibration.append(lst)
    return calibration

tt = {}
for var in ["PLATFORM_NUMBER", "LATITUDE", "LONGITUDE", "PRES", "PRES_ADJUSTED", "TEMP", "TEMP_ADJUSTED",
            "PSAL", "PSAL_ADJUSTED", "PROJECT_NAME"]: 
    '''
    ['HISTORY_INSTITUTION', 'HISTORY_STEP', 'HISTORY_SOFTWARE', 'HISTORY_SOFTWARE_RELEASE', 'HISTORY_REFERENCE',
    'HISTORY_DATE', 'HISTORY_ACTION', 'HISTORY_PARAMETER', 'HISTORY_START_PRES', 'HISTORY_STOP_PRES', 
    'HISTORY_PREVIOUS_VALUE', 'HISTORY_QCTEST'] (argo.variables.key()[52:]) are empty so we should not bother with them.
    '''
    if len(argo.variables[var][:]) <= 16:
    # Deals with the first variables which have a single information in them e.g. ['DATA_TYPE', 'FORMAT_VERSION', 'HANDBOOK_VERSION', 'REFERENCE_DATE_TIME', 'DATE_CREATION', 'DATE_UPDATE']
        tt[var] = ''.join(utf_decoding(argo.variables[var][:])).strip()
    elif argo.variables[var][:].dtype != '|S1' and argo.variables[var][:].ndim == 1:
        tt[var] = argo.variables[var][:]
    elif argo.variables[var][:].dtype != '|S1' and argo.variables[var][:].ndim > 1:
        tt[var] = list(argo.variables[var][:].data)
    elif argo.variables[var][:].ndim == 1:
        tt[var] = utf_decoding(argo.variables[var][:])
    elif argo.variables[var][:].ndim == 2:
        tt[var] = masked_arrays_decoding(argo.variables[var][:])
    elif argo.variables[var][:].ndim == 3:
        tt[var] = [masked_arrays_decoding(x) for x in argo.variables[var][:]]
    else:
        tt[var] = param_masked_arrays_decoding(argo.variables[var][:])
    
    

argo_df = pd.DataFrame(tt)
argo_df
latitude_limits = (argo_df["LATITUDE"] > 38) & (argo_df["LATITUDE"] < 59)
longitude_limits = (argo_df["LONGITUDE"] > -70) & (argo_df["LONGITUDE"] < -35)
argo_df[latitude_limits & longitude_limits]

# #Restrict floats to region surrounding the St Lawrence Estuary. This is a fairly large area but can be further filtered if needed. 
# #Keep only columns we will need and rename columns that will be retained after processing. 
argo_df = (argo_df[latitude_limits & longitude_limits]
           [["PLATFORM_NUMBER", "LATITUDE", "LONGITUDE", "PRES", "PRES_ADJUSTED", "TEMP", "TEMP_ADJUSTED", 
             "PSAL", "PSAL_ADJUSTED", "PROJECT_NAME"]]
           .rename(columns={"PLATFORM_NUMBER":"float_id", "LATITUDE":"latitude", "LONGITUDE":"longitude", "PROJECT_NAME": "data_source"})
          )
           
argo_df

,float_id,latitude,longitude,PRES,PRES_ADJUSTED,TEMP,TEMP_ADJUSTED,PSAL,PSAL_ADJUSTED,data_source
34,5903398,47.663000,-38.477000,"[5.8, 9.8, 19.6, 30.0, 40.2, 49.9, 59.9, 69.6,...","[5.8, 9.8, 19.6, 30.0, 40.2, 49.9, 59.9, 69.6,...","[12.169, 12.167, 12.152, 12.152, 12.159, 12.16...","[12.169, 12.167, 12.152, 12.152, 12.159, 12.16...","[35.272, 35.271, 35.271, 35.272, 35.273, 35.27...","[35.27204, 35.271088, 35.27102, 35.271988, 35....",US ARGO PROJECT
57,6900491,51.610000,-37.015000,"[16.0, 27.0, 36.0, 45.0, 55.0, 66.0, 76.0, 86....","[16.0, 27.0, 36.0, 45.0, 55.0, 66.0, 76.0, 86....","[8.813, 8.816, 8.861, 8.914, 8.931, 8.913, 8.9...","[8.813, 8.816, 8.861, 8.914, 8.931, 8.913, 8.9...","[34.408, 34.408, 34.422, 34.453, 34.468, 34.47...","[34.397907, 34.397907, 34.4119, 34.442886, 34....",OVIDE
80,4901104,43.945999,-37.765991,"[4.2, 9.2, 19.2, 29.2, 39.1, 49.3, 59.5, 69.4,...","[4.2, 9.2, 19.2, 29.2, 39.1, 49.3, 59.5, 69.4,...","[14.985, 14.986, 14.979, 14.976, 14.974, 14.96...","[14.985, 14.986, 14.979, 14.976, 14.974, 14.96...","[35.486, 35.486, 35.484, 35.482, 35.482, 35.48...","[35.48599, 35.485996, 35.484, 35.481983, 35.48...",Canadian Argo
81,4901151,51.707001,-36.863998,"[6.5, 10.1, 20.6, 30.0, 39.7, 49.6, 59.9, 67.6...","[6.7, 10.3, 20.800001, 30.2, 39.9, 49.8, 60.10...","[8.589, 8.594, 8.591, 8.624, 8.669, 8.515, 7.9...","[8.589, 8.594, 8.591, 8.624, 8.669, 8.515, 7.9...","[34.429, 34.429, 34.429, 34.445, 34.458, 34.55...","[34.428898, 34.4289, 34.428905, 34.4449, 34.45...",Argo Canada
82,4901152,56.067001,-44.601002,"[5.9, 9.7, 19.3, 30.2, 39.6, 50.0, 57.3, 70.5,...","[6.1, 9.9, 19.5, 30.400002, 39.8, 50.2, 57.5, ...","[4.091, 4.094, 4.095, 4.095, 4.096, 4.098, 4.0...","[4.091, 4.094, 4.095, 4.095, 4.096, 4.098, 4.0...","[34.707, 34.707, 34.706, 34.707, 34.707, 34.70...","[34.706886, 34.706886, 34.705902, 34.70689, 34...",Canadian Argo
83,4901153,57.682999,-46.846001,"[5.9, 10.1, 20.0, 30.2, 40.5, 50.0, 60.6, 70.0...","[6.6, 10.8, 20.7, 30.900002, 41.2, 50.7, 61.3,...","[4.739, 4.74, 4.74, 4.744, 4.745, 4.745, 4.742...","[4.739, 4.74, 4.74, 4.744, 4.745, 4.745, 4.742...","[34.889, 34.887, 34.889, 34.888, 34.888, 34.88...","[34.889, 34.887, 34.889, 34.888, 34.888, 34.88...",Canadian Argo
84,4901154,48.507999,-44.855999,"[6.4, 9.9, 19.8, 30.3, 40.2, 47.6, 60.0, 70.8,...","[6.4, 9.9, 19.8, 30.3, 40.2, 47.6, 60.0, 70.8,...","[6.456, 6.457, 6.458, 6.46, 6.462, 6.463, 6.45...","[6.456, 6.457, 6.458, 6.46, 6.462, 6.463, 6.45...","[34.095, 34.095, 34.094, 34.096, 34.097, 34.09...","[34.095005, 34.09501, 34.094006, 34.09601, 34....",Argo Canada
85,4901159,57.043999,-52.349998,"[6.1, 10.4, 19.6, 30.4, 40.3, 50.1, 60.4, 70.2...","[6.1, 10.4, 19.6, 30.4, 40.3, 50.1, 60.4, 70.2...","[3.767, 3.781, 3.781, 3.794, 3.788, 3.787, 3.8...","[3.767, 3.781, 3.781, 3.794, 3.788, 3.787, 3.8...","[34.627, 34.627, 34.627, 34.627, 34.627, 34.62...","[34.627003, 34.627007, 34.626995, 34.627, 34.6...",Canadian Argo
86,4901161,56.963001,-56.492001,"[6.5, 9.9, 20.3, 30.0, 40.4, 49.8, 60.0, 70.4,...","[6.7, 10.099999, 20.5, 30.2, 40.600002, 50.0, ...","[3.31, 3.311, 3.319, 3.318, 3.317, 3.318, 3.33...","[3.31, 3.311, 3.319, 3.318, 3.317, 3.318, 3.33...","[34.385, 34.385, 34.384, 34.385, 34.385, 34.38...","[34.385, 34.385, 34.384, 34.385, 34.385, 34.38...",Argo Canada


In [26]:
def unnesting(df, columns):
    '''Unnest the parameter data for Pressure, Salinity and Temperature. Pressure will be used to calculate depth.'''
    idx = df.index.repeat(df[columns[0]].str.len())
    df1 = pd.concat([pd.DataFrame({x: np.concatenate(df[x].values)}) for x in columns], axis=1)
    df1.index = idx

    return df1.join(df.drop(columns, 1), how='left')

def param(columns):
    '''Removes NaN values (99999.00) and select measured parameter when the adjusted parameters is NaN.'''
    non_adjusted = columns[0]
    adjusted = columns[1]
    param = []
    for x in range(len(columns[0])):
        if adjusted.iloc[x] >= 99999:
            param.append(non_adjusted.iloc[x])
        elif adjusted.iloc[x] >= 99999:
            param.append("NaN")
        else:
            param.append(non_adjusted.iloc[x])
    return param

to_unnest = ['PRES', 'PRES_ADJUSTED','PSAL', 'PSAL_ADJUSTED', 'TEMP', 'TEMP_ADJUSTED']       

argo_df["data_date"] = pd.to_datetime("20120104")
unnested_argo = unnesting(argo_df, to_unnest)
# print(len(unnested_argo[unnested_argo["PRES"] != 99999].dropna()))
# print(len(unnested_argo[unnested_argo["PSAL"] != 99999].dropna()))
# print(len(unnested_argo[unnested_argo["TEMP"] != 99999].dropna()))
unnested_argo["temperature"] = param([unnested_argo["TEMP"], unnested_argo["TEMP_ADJUSTED"]])
unnested_argo
unnested_argo["salinity"] = param([unnested_argo["PSAL"], unnested_argo["PSAL_ADJUSTED"]])
# For every 10 meters you go down, the pressure increases by one atmosphere (approx 1 decibar) so with pressure recorded
# in decibar we can assume depth is equivalent.
unnested_argo["depth"] = param([unnested_argo["PRES"], unnested_argo["PRES_ADJUSTED"]])
t = unnested_argo[["data_date", "float_id", "latitude", "longitude", "depth", "temperature", "salinity"]]
t = t[t != 99999].dropna()
t

,data_date,float_id,latitude,longitude,depth,temperature,salinity
34,2012-01-04,5903398,47.663000,-38.477000,5.800000,12.169,35.271999
34,2012-01-04,5903398,47.663000,-38.477000,9.800000,12.167,35.271000
34,2012-01-04,5903398,47.663000,-38.477000,19.600000,12.152,35.271000
34,2012-01-04,5903398,47.663000,-38.477000,30.000000,12.152,35.271999
34,2012-01-04,5903398,47.663000,-38.477000,40.200001,12.159,35.272999
...,...,...,...,...,...,...,...
86,2012-01-04,4901161,56.963001,-56.492001,1800.199951,3.194,34.914001
86,2012-01-04,4901161,56.963001,-56.492001,1850.000000,3.142,34.912998
86,2012-01-04,4901161,56.963001,-56.492001,1900.199951,3.102,34.912998
86,2012-01-04,4901161,56.963001,-56.492001,1949.900024,3.038,34.912998


In [27]:
bins = range(0,int(t["depth"].max()+100),100)
#t['binned'] = pd.cut(t['depth'], bins)
t = t.groupby(["data_date", "float_id", pd.cut(t.depth, bins)]).mean().drop("depth", axis=1).reset_index().dropna()
t

,data_date,float_id,depth,latitude,longitude,temperature,salinity
0,2012-01-04,4901104,"(0, 100]",43.945999,-37.765991,14.990636,35.508272
1,2012-01-04,4901104,"(100, 200]",43.945999,-37.765991,14.309900,35.625000
2,2012-01-04,4901104,"(200, 300]",43.945999,-37.765991,13.690400,35.639800
3,2012-01-04,4901104,"(300, 400]",43.945999,-37.765991,12.619125,35.556750
4,2012-01-04,4901104,"(400, 500]",43.945999,-37.765991,11.087500,35.384001
...,...,...,...,...,...,...,...
183,2012-01-04,6900491,"(1500, 1600]",51.610000,-37.015000,3.638000,34.924749
184,2012-01-04,6900491,"(1600, 1700]",51.610000,-37.015000,3.570500,34.924999
185,2012-01-04,6900491,"(1700, 1800]",51.610000,-37.015000,3.496250,34.924000
186,2012-01-04,6900491,"(1800, 1900]",51.610000,-37.015000,3.429000,34.922251


In [28]:
list(t.depth.unique().astype(str))

['(0, 100]',
 '(100, 200]',
 '(200, 300]',
 '(300, 400]',
 '(400, 500]',
 '(500, 600]',
 '(600, 700]',
 '(700, 800]',
 '(800, 900]',
 '(900, 1000]',
 '(1000, 1100]',
 '(1100, 1200]',
 '(1200, 1300]',
 '(1300, 1400]',
 '(1400, 1500]',
 '(1500, 1600]',
 '(1600, 1700]',
 '(1700, 1800]',
 '(1800, 1900]',
 '(1900, 2000]',
 '(2000, 2100]']

In [317]:
t["data_source"] = "Argo Project"
t.to_csv("one_month.csv")

In [297]:
t["depth"] = t["depth"].astype(str)
t["data_date"] = pd.to_datetime(t["data_date"])
t#.dtypes

,data_date,float_id,latitude,longitude,depth,temperature,salinity
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,5.0,17.392000,36.308998
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,10.0,17.393999,36.308998
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,15.0,17.391001,36.306999
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,20.0,17.393999,36.306999
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,25.0,17.395000,36.307999
...,...,...,...,...,...,...,...
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1800.0,3.523000,34.932999
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1850.4000244140625,3.478000,34.932999
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1899.699951171875,3.458000,34.932999
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1949.300048828125,3.422000,34.931000


In [293]:
t["data_date"] = pd.to_datetime(t["data_date"])


,data_date,float_id,latitude,longitude,depth,temperature,salinity
26,2020-09-03,4901218,41.652000,-41.517000,5.0,17.392000,36.308998
26,2020-09-03,4901218,41.652000,-41.517000,10.0,17.393999,36.308998
26,2020-09-03,4901218,41.652000,-41.517000,15.0,17.391001,36.306999
26,2020-09-03,4901218,41.652000,-41.517000,20.0,17.393999,36.306999
26,2020-09-03,4901218,41.652000,-41.517000,25.0,17.395000,36.307999
...,...,...,...,...,...,...,...
114,2020-09-03,4901168,51.250999,-38.369999,1800.0,3.523000,34.932999
114,2020-09-03,4901168,51.250999,-38.369999,1850.4000244140625,3.478000,34.932999
114,2020-09-03,4901168,51.250999,-38.369999,1899.699951171875,3.458000,34.932999
114,2020-09-03,4901168,51.250999,-38.369999,1949.300048828125,3.422000,34.931000


In [294]:
t.dtypes

data_date      datetime64[ns]
float_id               object
latitude              float64
longitude             float64
depth                  object
temperature           float64
salinity              float64
dtype: object

In [10]:
# TODO: decide whether expanding all parameters data before or after writing to DB. Param data contains a lot of noise (e.g. 99999.0000 values) 
# Is depth indicated somewhere? or all measures taken at specific depth? TBC from documentation

In [51]:
floats = pd.read_csv('floats.csv')
floats = floats['row'].str[1:-1].str.split(',', expand=True).astype(float).rename(columns={0:"latitude", 1:"longitude"})
floats.head()

,latitude,longitude
0,38.0,-68.9
1,38.0,-68.5
2,38.0,-61.5
3,38.0,-59.7
4,38.0,-56.3


In [52]:
import folium
m = folium.Map(location=[50, -60], zoom_start=4.5, tiles='Stamen Terrain')

for flt in floats.index:
    folium.CircleMarker(location=[floats["latitude"].iloc[flt], floats["longitude"].iloc[flt]], radius=2, fill=True).add_to(m)

m

In [12]:
temp_df = argo_df[['TEMP']].copy()
temp_df

,TEMP
0,"[26.675, 26.67, 26.663, 26.662, 26.671, 26.669..."
1,"[27.102, 27.103, 27.103, 27.104, 27.104, 27.10..."
2,"[23.352, 23.352, 23.349, 23.348, 23.355, 23.35..."
3,"[23.377, 23.377, 23.382, 23.392, 23.395, 23.39..."
4,"[19.215, 19.229, 19.234, 19.233, 19.23, 19.226..."
...,...
137,"[4.478, 4.476, 4.475, 4.476, 4.473, 4.475, 4.4..."
138,"[4.35, 4.349, 4.35, 4.352, 4.352, 4.352, 4.352..."
139,"[3.674, 3.68, 3.689, 3.691, 3.691, 3.692, 3.69..."
140,"[14.644, 14.665, 14.678, 14.675, 14.678, 14.68..."


In [13]:
df_temp_test = pd.DataFrame(temp_df['TEMP'].to_list())
df_temp_test = df_temp_test.T.replace(99999.0000, None)
# A lot of temperature lists include 99999.0000 values which we need to get rid of
for col in df_temp_test.columns:
    print(f'{col}: min {df_temp_test[col].min()}, max {df_temp_test[col].max()}, mean {df_temp_test[col].mean()}')

0: min 5.315000057220459, max 26.690000534057617, mean 6.271593895368071
1: min 3.5369999408721924, max 27.106000900268555, mean 4.702252833413354
2: min 3.6659998893737793, max 23.368000030517578, mean 5.499650800285495
3: min 3.703000068664551, max 23.399999618530273, mean 5.5425627958946695
4: min 3.7829999923706055, max 19.236000061035156, mean 5.517728609493246
5: min 3.86899995803833, max 25.993000030517578, mean 5.412251720689747
6: min 2.447999954223633, max 9.5649995803833, mean 2.8920603186852047
7: min 3.746999979019165, max 18.350000381469727, mean 4.848009606106817
8: min 3.571000099182129, max 28.097999572753906, mean 4.861038892545421
9: min 0.9089999794960022, max 27.136999130249023, mean 3.3207352862716473
10: min 14.824000358581543, max 23.94300079345703, mean 14.946489790641392
11: min 3.8889999389648438, max 23.940000534057617, mean 5.006264294310439
12: min 5.249000072479248, max 24.63599967956543, mean 6.232784584600455
13: min 1.7790000438690186, max 25.597999572

In [14]:
temp_df = argo_df[['LATITUDE', 'LONGITUDE']]
temp_df['MEAN_TPT'] = pd.Series([df_temp_test[col].mean() for col in df_temp_test.columns])
temp_df

C:\Users\ssung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LATITUDE,LONGITUDE,MEAN_TPT
0,9.201430,-37.727650,6.271594
1,-6.972160,-9.659960,4.702253
2,19.470060,-30.365670,5.499651
3,19.456280,-30.361650,5.542563
4,22.753110,-18.078970,5.517729
...,...,...,...
137,54.006447,-50.540653,3.459037
138,58.633392,-39.617931,3.436786
139,57.910954,-54.497654,3.471710
140,41.499115,-49.142590,3.663514


In [15]:
from folium.plugins import HeatMap
HeatMap(data=temp_df[['LATITUDE', 'LONGITUDE', 'MEAN_TPT']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), radius=25, max_zoom=10).add_to(m)
m

In [ ]:
# TODO: continue looking at https://shapely.readthedocs.io/en/latest/manual.html to find ways to divide multipolygon files into sub-areas. Seem to not be trivial outside of ArcGIS/QGIS.